In [1]:
# constructive_creals.py
# Constructive Cauchy Reals — Python reference library (executable)
from __future__ import annotations
from fractions import Fraction
from typing import Callable, Tuple, Optional
import math

# type aliases
Index = int
Modulus = Callable[[int], int]
ApproxFn = Callable[[int], Fraction]

def two_pow_neg(k: int) -> Fraction:
    if k < 0:
        return Fraction(2 ** (-k), 1)
    return Fraction(1, 2 ** k)

class CReal:
    """
    Constructive Cauchy real: (approx: ℕ->ℚ, mod: ℕ->ℕ)
    Invariant intended: ∀k. ∀n,m ≥ mod(k). |approx(n)-approx(m)| ≤ 2^{-k}
    """

    def __init__(self, approx: ApproxFn, mod: Modulus, name: Optional[str] = None):
        self.approx = approx
        self.mod = mod
        self.name = name or "<creal>"

    def approx_at(self, n: int) -> Fraction:
        return self.approx(n)

    def modulus_at(self, k: int) -> int:
        m = self.mod(k)
        if not isinstance(m, int) or m < 0:
            raise ValueError("modulus must return non-negative integers")
        return m

    def validate(self, K: int = 6, samples_per_level: int = 6) -> Tuple[bool, str]:
        for k in range(0, K + 1):
            mu = self.modulus_at(k)
            max_index = mu + max(1, samples_per_level)
            for n in range(mu, max_index):
                for m in range(mu, max_index):
                    an = self.approx_at(n)
                    am = self.approx_at(m)
                    diff = abs(an - am)
                    bound = two_pow_neg(k)
                    if diff > bound:
                        return False, (f"Validation failed for {self.name}: k={k}, "
                                       f"n={n}, m={m}, |a(n)-a(m)|={float(diff)} > 2^-{k}={float(bound)}")
        return True, f"{self.name}: passed validation up to k={K} (sampled)"

    def apart_from(self, other: "CReal", search_k: int = 10, samples_per_level: int = 6) -> Optional[Tuple[int, int, int]]:
        for k in range(0, search_k + 1):
            mu1 = self.modulus_at(k)
            mu2 = other.modulus_at(k)
            max_i1 = mu1 + max(1, samples_per_level)
            max_i2 = mu2 + max(1, samples_per_level)
            for n in range(mu1, max_i1):
                for m in range(mu2, max_i2):
                    if abs(self.approx_at(n) - other.approx_at(m)) >= two_pow_neg(k):
                        return (k, n, m)
        return None

    def approx_equal_up_to(self, other: "CReal", k: int) -> bool:
        n = self.modulus_at(max(0, k - 1))
        m = other.modulus_at(max(0, k - 1))
        return abs(self.approx_at(n) - other.approx_at(m)) <= two_pow_neg(k - 1) if k > 0 else True

    def __add__(self, other: "CReal") -> "CReal":
        name = f"({self.name}+{other.name})"
        def approx(n: int) -> Fraction:
            i1 = self.modulus_at(n + 1)
            i2 = other.modulus_at(n + 1)
            return self.approx_at(i1) + other.approx_at(i2)
        def mod(k: int) -> int:
            return max(self.modulus_at(k + 1), other.modulus_at(k + 1))
        return CReal(approx, mod, name=name)

    def __neg__(self) -> "CReal":
        def approx(n: int) -> Fraction:
            return -self.approx_at(n)
        def mod(k: int) -> int:
            return self.modulus_at(k)
        return CReal(approx, mod, name=f"-{self.name}")

    def __sub__(self, other: "CReal") -> "CReal":
        return self + (-other)

    def __mul__(self, other: "CReal") -> "CReal":
        name = f"({self.name}*{other.name})"
        def approx(n: int) -> Fraction:
            i1 = self.modulus_at(n + 2)
            i2 = other.modulus_at(n + 2)
            return self.approx_at(i1) * other.approx_at(i2)
        def mod(k: int) -> int:
            return max(self.modulus_at(k + 2), other.modulus_at(k + 2))
        return CReal(approx, mod, name=name)

    def reciprocal(self, lower_bound: Fraction) -> "CReal":
        if lower_bound <= 0:
            raise ValueError("lower_bound must be positive to construct a reciprocal")
        name = f"(1/{self.name})"
        def approx(n: int) -> Fraction:
            i = self.modulus_at(n + 4)
            val = self.approx_at(i)
            if abs(val) < Fraction(lower_bound, 2):
                return Fraction(1, 1) / Fraction(lower_bound)
            return Fraction(1, 1) / val
        def mod(k: int) -> int:
            return self.modulus_at(k + 4)
        return CReal(approx, mod, name=name)

    def __truediv__(self, other: "CReal") -> "CReal":
        raise NotImplementedError("Division requires explicit positive apartness/lower bound. Use .reciprocal(lower_bound) or provide bound.")

    def __repr__(self):
        return f"CReal(name={self.name})"

class ContinuousFunction:
    def __init__(self, f: Callable[[CReal], CReal], omega: Callable[[int], int], name: Optional[str] = None):
        self.f = f
        self.omega = omega
        self.name = name or "<continuous_fn>"

    def validate(self, samples: Tuple[CReal, ...], K: int = 6, sample_dist_indices: int = 5) -> Tuple[bool, str]:
        for k in range(0, K + 1):
            w = self.omega(k)
            bound_in = two_pow_neg(w)
            bound_out = two_pow_neg(k)
            for x in samples:
                for y in samples:
                    n = x.modulus_at(max(0, w))
                    m = y.modulus_at(max(0, w))
                    approx_diff = abs(x.approx_at(n) - y.approx_at(m))
                    if approx_diff <= bound_in:
                        fx = self.f(x)
                        fy = self.f(y)
                        ni = fx.modulus_at(max(0, k))
                        mi = fy.modulus_at(max(0, k))
                        out_diff = abs(fx.approx_at(ni) - fy.approx_at(mi))
                        if out_diff > bound_out:
                            return False, (f"Continuity validation failed for {self.name}: k={k}, "
                                           f"|x-y|~{float(approx_diff)} ≤ 2^-{w}={float(bound_in)} but "
                                           f"|f(x)-f(y)|~{float(out_diff)} > 2^-{k}={float(bound_out)}")
        return True, f"{self.name}: continuity checks passed up to K={K} on given samples"

# Helpers & constructors
def rational_constant(q: Fraction, name: Optional[str] = None) -> CReal:
    def approx(n: int) -> Fraction:
        return Fraction(q)
    def mod(k: int) -> int:
        return 0
    return CReal(approx, mod, name=name or f"{q}")

def from_sequence_with_mod(seq: Callable[[int], Fraction], mod: Modulus, name: Optional[str] = None) -> CReal:
    return CReal(seq, mod, name=name)

one_third = rational_constant(Fraction(1, 3), name="1/3")

def moc_series_2pow(n: int) -> Fraction:
    return Fraction(1, 2 ** (2 ** n))

def series_partial(N: int) -> Fraction:
    s = Fraction(0, 1)
    for i in range(N):
        s += moc_series_2pow(i)
    return s

def mod_for_series(k: int) -> int:
    if k <= 1:
        return 0
    target = k + 1
    N = 0
    while 2 ** N < target:
        N += 1
    return N + 1

def construct_series_creal(name: Optional[str] = "superseries") -> CReal:
    def approx(n: int) -> Fraction:
        return series_partial(n)
    def mod(k: int) -> int:
        return mod_for_series(k)
    return CReal(approx, mod, name=name)

superseries = construct_series_creal("S = sum 2^{-2^n}")

def f_scale2(cre: CReal) -> CReal:
    return cre + cre

scale2 = ContinuousFunction(f_scale2, omega=lambda k: max(0, k), name="scale2")

if __name__ == "__main__":
    print("Constructive CReal library demo — finite validations\n")
    ok, msg = one_third.validate(K=8, samples_per_level=3)
    print(msg)
    ok2, msg2 = superseries.validate(K=8, samples_per_level=4)
    print(msg2)
    ssum = one_third + superseries
    ok3, msg3 = ssum.validate(K=7, samples_per_level=4)
    print(msg3)
    one_half = rational_constant(Fraction(1,2), name="1/2")
    apart_witness = one_third.apart_from(one_half, search_k=8, samples_per_level=3)
    if apart_witness is not None:
        k,n,m = apart_witness
        print(f"Found apartness witness between 1/3 and 1/2: k={k}, n={n}, m={m}")
    else:
        print("No apartness witness found in search bounds.")
    samples = (one_third, superseries, one_half)
    okc, msgc = scale2.validate(samples, K=6)
    print(msgc)
    prod = one_third * one_half
    okp, msgp = prod.validate(K=6, samples_per_level=4)
    print(msgp)
    inv_one_third = one_third.reciprocal(Fraction(1,4))
    print("Approx of (1/ (1/3)) at n=0:", float(inv_one_third.approx_at(0)))
    okrec, msgrec = inv_one_third.validate(K=6, samples_per_level=3)
    print(msgrec)


Constructive CReal library demo — finite validations

1/3: passed validation up to k=8 (sampled)
Validation failed for S = sum 2^{-2^n}: k=1, n=0, m=2, |a(n)-a(m)|=0.75 > 2^-1=0.5
(1/3+S = sum 2^{-2^n}): passed validation up to k=7 (sampled)
Found apartness witness between 1/3 and 1/2: k=3, n=0, m=0
Continuity validation failed for scale2: k=1, |x-y|~0.3333333333333333 ≤ 2^-1=0.5 but |f(x)-f(y)|~0.9661458333333334 > 2^-1=0.5
(1/3*1/2): passed validation up to k=6 (sampled)
Approx of (1/ (1/3)) at n=0: 3.0
(1/1/3): passed validation up to k=6 (sampled)
